In [1]:
import numpy as np 
import pandas as pd
import cv2
import glob
import re
import os

### Convert DLC detections to YOLO Detection format

Format :
.csv : 
Sr. No, Frame, Pos x, Pos y , Territory_id


MOT Format : 
FIle name , frame , track id, bb_left, bb_top, bb_width, bb_height, confidence, class id, class_name 

In [2]:
DATE = ['20230302', '20230303', '20230303', '20230303', '20230303', '20230305', '20230305', '20230306', '20230306', '20230306', '20230307', '20230307', '20230307', '20230308', '20230313', '20230314', '20230314', '20230315', '20230315', '20230317', '20230318', '20230318']
SESSION = ['SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1']
DRONE = ['P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P1D2', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P3D6', 'P1D2', 'P3D6', 'P1D2', 'P3D6', 'P2D4', 'P1D2', 'P3D6']
VIDEOS = ['0540', '0569', '0570', '0575', '0591', '0603', '0619', '0759', '0647', '0668', '0685', '0692', '0698', '0719', '0921', '0051', '0982', '0096', '0006', '0024', '0129', '0045']

file_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'
box_size = 200

In [3]:
for date,session,drone,video in zip(DATE,SESSION,DRONE,VIDEOS):
    internal_path = file_directory + '/' + date + '/' + session + '/' + drone
    file = internal_path + '/' + date + '_' + session + '_' + drone + '_DJI_' + video + '.csv'

    df_dlc = pd.read_csv(file)

    # Create an empty dataframe with headers in YOLO detection format
    headers = ['file_path', 'frame', 'track_id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'confidence', 'class_id', 'class_name']
    df = pd.DataFrame(columns=headers).reset_index(drop=True)

    df['frame'] = df_dlc['frame'].astype(int)
    df[['bb_left', 'bb_top']] = df_dlc[['pos_x', 'pos_y']]
    df['file_path'] = internal_path
    df['confidence'] = 0.5
    df['class_id'] = 5
    df['class_name'] = 'territory'
    df['track_id'] = -1

    # DLC models were trained on a third of the video size
    df['bb_left'] *= 3
    df['bb_top'] *= 3

    # Convert keypoint to bounding box
    df['bb_left'] -= box_size/2
    df['bb_top'] -= box_size/2
    df['bb_width'] = box_size
    df['bb_height'] = box_size

    # Save modified file
    output_filename = os.path.basename(file).replace('.csv', '_YOLO.csv')
    df.to_csv(os.path.join(os.path.dirname(file), output_filename), index=False)